### Purged K-Fold Cross Validation

정보 누출을 감소시키는 방법 중 하나는 훈련 데이터셋에서 현재 테스트세트와 레이블이 중첩된 모든 관측값을 제거하는 것이다. Prado 교수는 이 절차를 Purging이라고 불렀다. 게다가 금융데이터는 대부분 시계열적이므로 자기상관을 보이는 시계열을 종종 포함한다. 테스트셋에 있는 관측값을 즉시 따르는 훈련셋 관측값을 제거해야 한다. 저자인 Prado교수는 이것을 Embargo process라고 부른다

In [1]:
import numpy as np
import pandas as pd
from FinancialMachineLearning.labeling.labeling import *

triple_barrier_event = pd.read_parquet('../../../Data/AAPL_triple_barrier_events.parquet')
avg_uniqueness = pd.read_parquet('../../../Data/AAPL_avg_unique.parquet')
feature_matrix = pd.read_parquet('../../../Data/AAPL_feature_matrix.parquet')

labels = meta_labeling(
    triple_barrier_event, 
    feature_matrix['Close']
)

triple_barrier_event['side'] = labels['bin']
meta_labels = meta_labeling(
    triple_barrier_event, # with side labels
    feature_matrix['Close']
)

In [2]:
feature_matrix['side'] = triple_barrier_event['side'].copy()
feature_matrix['label'] = meta_labels['bin'].copy()
feature_matrix.drop(['Open','High','Low','Close','Adj Close','Volume'], axis = 1, inplace = True)
feature_matrix.dropna(inplace = True)
matrix = feature_matrix[feature_matrix['side'] != 0]

X = matrix.drop(['side','label'], axis = 1)
y = matrix['label']

X_train, X_test = X.loc[:'2019'], X.loc['2020':]
y_train, y_test = y.loc[:'2019'], y.loc['2020':]

In [3]:
from sklearn.ensemble import RandomForestClassifier
from FinancialMachineLearning.sample_weights.bootstrapping import *

class SequentialRandomForestClassifier(RandomForestClassifier):
    def _generate_sample_indices(self, random_state, n_samples):
        """Generate bootstrap sample indices with sequential bootstrap method."""
        random_instance = random_state  # get the RandomState instance
        
        ind_mat = get_indicator_matrix(
            triple_barrier_event.index.to_series(),
            triple_barrier_event['t1']
        )
        
        sample_indices = seq_bootstrap(ind_mat, n_samples)
        
        return sample_indices

In [4]:
forest = SequentialRandomForestClassifier(
    criterion = 'entropy',
    class_weight = 'balanced_subsample',
    random_state = 42,
    n_estimators = 100,
    max_features = 3, # early stopping
    min_weight_fraction_leaf = 0.05, # early stopping
    oob_score = True
)

forest_fit = forest.fit(
    X = X_train, 
    y = y_train, 
    sample_weight = avg_uniqueness.loc[X_train.index].to_numpy().reshape(1, -1)[0]
)

#### Purging the Training Set

label이 $Y_j$인 테스트 관측값이 정보 집합 $\Phi_j$에 근거해 결정됐다고 가정해 보자. 정보의 누출 형태를 방지하고자 정보 집합 $\Phi_i$에 근거해서 label $Y_i$를 결정한 모든 관측값을 훈련 데이터에서 제거해서 $\Phi_i \cap \Phi_j = \emptyset$이 되게 한다. 즉, 테스트셋에 정보의 Concurrency가 존재하는 데이터를 훈련 데이터셋에서 삭제하는 과정이다.

특별히 $Y_i$와 $Y_j$가 공존할 때에는 언제나 양쪽 label이 적어도 하나의 공통 무작위 추출에 달려 있다는 관점에서 두 관측값 $i$와 $j$ 사이에 정보의 중첩이 있다고 결정한다. 예를 들어서, 닫힌 구간 $t \in [t_{j,0}, t_{j,1}], Y_j = f\left[ [t_{j,0}, t_{j,1}]\right]$에서 관측값의 함수인 label $Y_j$를 고려해 보자. 예를 들어서, triple barrier labeling의 관점에서는 label이 인덱스가 $t_{j,0}$와 $t_{j,1}$ 사이의 가격 바 수익률의 부호, 즉 $\mathrm{sgn}[r_{t_{j, 0}, t_{j, 1}}]$이다. label $Y_i = f\left[ [t_{i,0}, t_{i,1}]\right]$은 다음 세 가지 충분 조건 중 하나가 만족되면 $Y_j$와 중첩된다

1. $t_{j,0} \leq t_{i,0} \leq t_{j,1}$
2. $t_{j,0} \leq t_{i,1} \leq t_{j,1}$
3. $t_{i,0} \leq t_{j,0} \leq t_{j,1} \leq t_{i,1}$

아래의 코드는 훈련 데이터셋에서 관측값 제거를 구현한다. 처음과 마지막 테스트 관측값 사이에서는 훈련 관측값이 발생하지 않는다는 관점에서 테스트셋이 연속이면 제거는 가속화될 수 있다.

In [5]:
from FinancialMachineLearning.cross_validation.cross_validation import get_train_times, get_embargo_times

train_times, test_times = triple_barrier_event.loc[:'2019'], triple_barrier_event.loc['2019':]

In [6]:
train_times = get_train_times(train_times['t1'], test_times['t1'])

In [7]:
train_times.head()

Date
2000-01-07   2000-01-11
2000-01-10   2000-01-12
2000-01-11   2000-01-12
2000-01-12   2000-01-13
2000-01-13   2000-01-19
Name: t1, dtype: datetime64[ns]

누출이 발생하면 단순히 $k \rightarrow T$로 증가시키는 것만으로도 성과 개선이 일어난다. 여기서 $T$는 바의 개수다. 그 이유는 테스트 분할 개수가 많을수록 훈련셋의 중첩된 개수가 많아지기 때문이다. 많은 경우 제거만으로 누수를 막을 수 있다. $k$를 증가시킬수록 모델을 더 자주 조율하게 되므로 성능은 더 향상된다. 그러나 특정 값 $k^*$이상으로는 성능이 더 이상 향상되지 않는데, 이는 백테스트가 누출로부터 혜택을 얻지 못함을 의미한다. 아래의 그림은 K-fold cross validation의 한 부분을 그린 것이다. 테스트셋은 2개의 훈련 데이터셋에 둘러싸여 있기 때문에, 누출을 막기 위해서는 두 개의 제거해야 할 중첩 구간이 생성된다

![concurrency](./images/concurrency.png)

#### Embargo

제거로도 정보 누출을 방지하지 못하는 경우에는 모든 테스트셋 다음의 훈련 관측값에게 embargo를 설정할 수 있다. 훈련 label $Y_i = f\left[ [t_{i,0}, t_{i,1}]\right]$은 $t_{i,1} < t_{j,0}$에서 테스트 시간 $t_{j,0}$에 있었던 정보를 포함하기 때문에 Embargo는 테스트셋 이전의 훈련 관측값에 대해 조치를 취하는 것이 아니다. 달리 말하면 여기서는 오직 테스트 직후 $t_{j,1} \leq t_{i,0} \leq t_{j,1} + h$에 발생하는 train label $Y_i = f\left[ [t_{i,0}, t_{i,1}]\right]$만 고려하기 때문이다.

이 embargo 기간 $h$는 제거 이전에 $Y_i = f\left[ [t_{j,0}, t_{j,1} + h]\right]$로 설정하면 구현할 수 있다. $k \rightarrow T$로 증가시켜도 테스트 성능이 무한정 향상되지 않는다는 것을 확인 가능한 바와 같이 작은 값 $h \approx 0.01T$는 대개 모든 정보 누출을 방지하기에 충분하다. 아래의 그림은 테스트 셋 직후에 훈련 관측값에 엠바고를 설정하는 것을 보여 준다.

![embargo](./images/embargo.png)

아래의 코드는 앞서 설명한 embargo process에 대해서 보여준다.

In [8]:
mbrg = get_embargo_times(
    times = test_times.index, 
    pct_embargo = 0.01
)

In [9]:
mbrg.head()

Date
2019-01-02   2019-01-22
2019-01-03   2019-01-23
2019-01-04   2019-01-24
2019-01-07   2019-01-25
2019-01-08   2019-01-28
dtype: datetime64[ns]

#### Purged K Fold Class

앞서 레이블이 중첩상태일 때 훈련 테스트 데이터 분할을 어떻게 생성할지 알아보았다. 모델 개발의 특서한 맥락에서 제거와 embargo 개념을 소개했다. 일반적으로 Hyper parameter fitting이나 Backtesting, 성과 평가 등에 관계없이 Train / Test data split을 생성할 때마다 중첩된 훈련 관측값을 제거하고 Embargo 해야 한다. 아래의 코드는 `scikit-learn`의 K Fold 클래스를 확장해 테스트 정보가 훈련셋으로 누출될 가능성을 막는 과정이다

In [10]:
from FinancialMachineLearning.cross_validation.cross_validation import PurgedKFold

purged_k_fold = PurgedKFold(
    n_splits = 10,
    samples_info_sets = triple_barrier_event['t1'].loc[X_train.index],
    pct_embargo = 0.01
)

### Cross Validation

`sklearn`의 cross validation 함수는 다음과 같은 버그가 존재한다고 알려져 있다.

1. scoring function은 `sklearn`의 `pandas.Series`가 아니라 `numpy` 배열로 구현되어 있기 때문에 `classes_`를 알지 못한다.
2. `cross_val_score`는 가중값을 `.fit()` method에 전달하지만 `log_loss`에는 전달하지 않으므로 다른 결과를 산출한다.

In [11]:
from FinancialMachineLearning.cross_validation.cross_validation import cross_val_score
from sklearn.metrics import log_loss

cross_validation_score = cross_val_score(
    classifier = forest,
    X = X_train,
    y = y_train,
    sample_weight = avg_uniqueness.loc[X_train.index].to_numpy().reshape(1, -1)[0],
    cv_gen = purged_k_fold,
    scoring = log_loss
)

In [12]:
cross_validation_score

array([-0.61606639, -0.57196965, -0.60610062, -0.5293604 , -0.58362231,
       -0.50837912, -0.53492057, -0.4985293 , -0.51140731, -0.53649262])

이런 버그가 있음을 인지하고, `sklearn`의 `cross_val_score`대신 위 함수를 사용하도록 하자